# Import libraries and dependecies

In [48]:
#import libraires and dependencies
import pandas as pd
from pathlib import Path
import hvplot.pandas

#import the path
csv_path = Path('./hotel_bookings_raw.csv')

#read csv into dataframe
hotel_booking_df = pd.read_csv(csv_path)

#displaying column names
hotel_booking_df.columns

#Rename columns to create a arrival date column with 
year = 'year'
hotel_booking_df = hotel_booking_df.rename(columns={'arrival_date_year': year})
month = 'month'
hotel_booking_df = hotel_booking_df.rename(columns={'arrival_date_month': month})
day = 'day'
hotel_booking_df = hotel_booking_df.rename(columns={'arrival_date_day_of_month': day})

#check to be sure of column name change
hotel_booking_df.head()

#creating a rgular arrival date column to make it easier to read and graph
#setting index to new arrival date column

hotel_booking_df['arrival_date'] = hotel_booking_df['month'].astype(str) + '/' + hotel_booking_df['day'].astype(str) + '/' + hotel_booking_df['year'].astype(str)
hotel_booking_df['arrival_date'] = pd.to_datetime(hotel_booking_df['arrival_date'], format='%B/%d/%Y')
hotel_booking_df.set_index('arrival_date', inplace=True)

#creating seperate df for resort hotel and city hotel
resort_hotel_df = hotel_booking_df.loc[hotel_booking_df['hotel'] == 'Resort Hotel']
city_hotel_df = hotel_booking_df.loc[hotel_booking_df['hotel'] == 'City Hotel']

#checking to make sure lists are seperated
#resort_hotel_df.head()
#city_hotel_df.head()

# Generate the averages for the Resort Hotels that have to do with cancelations and print.

In [49]:
#finding the average "is_canceled" , "previous_cancellations" , "previous_booking_not_canceled" , "booking_changes" , 
#for both the resort hotel and the city hotel  for comparison

resort_hotel_avg_cancels = resort_hotel_df["is_canceled"].mean()
#previous_cancellations_resort_hotel = resort_hotel_df["previous_cancellations"].mean()
#previous_booking_not_canceled_resort_hotel = resort_hotel_df["previous_bookings_not_canceled"].mean()
resort_hotel_booking_changes = resort_hotel_df ["booking_changes"].mean()

#printing 
print(f"The average cancel rate of the Resort Hotel is {resort_hotel_avg_cancels}.")
#print(f"The average previous cancellation rate of the Resort Hotel is {previous_cancellations_resort_hotel}.")
#print(f"The average previous bookings not cancelled rate of the Resort Motel is {previous_booking_not_canceled_resort_hotel}.")
print(f"The average amount of booking changes of the Resort Hotel is {resort_hotel_booking_changes}.")

The average cancel rate of the Resort Hotel is 0.27763354967548676.
The average amount of booking changes of the Resort Hotel is 0.2879680479281078.


# Generate the averages for the City Hotel that have to do with cancelations and print.

In [50]:
city_hotel_avg_cancels = city_hotel_df["is_canceled"].mean()
#previous_cancellations_city_hotel = city_hotel_df["previous_cancellations"].mean()
#precious_booking_not_canceled_city_hotel = city_hotel_df["previous_bookings_not_canceled"].mean()
city_hotel_booking_changes = city_hotel_df ["booking_changes"].mean()

print(f"The average cancel rate of the City Hotel is {city_hotel_avg_cancels}.")
#print(f"The average previous cancellation rate of the City Hotel is {previous_cancellations_city_hotel}.")
#print(f"The average previous bookings not cancelled rate of the City Hotel is {precious_booking_not_canceled_city_hotel}.")
print(f"The average amount of booking changes of the City Hotel is {city_hotel_booking_changes}.")

The average cancel rate of the City Hotel is 0.41726963317786464.
The average amount of booking changes of the City Hotel is 0.18736921719399974.


# Who has the higher cancellation rate between the two?

In [51]:
#can you add the numbers in the output string?
if (city_hotel_avg_cancels) > (resort_hotel_avg_cancels):
    print("The City Hotel has a higher cancellation rate than the Resort Hotel")
elif (resort_hotel_avg_cancels) > (city_hotel_avg_cancels):
    print("The Resort Hotel has a higher cancellation rate than the City Hotel")
else:
    print("The cancellation rates for both hotels are equal")    

The City Hotel has a higher cancellation rate than the Resort Hotel


# Create a database for average cancels of both Hotels.

In [52]:
cancelation_df = hotel_booking_df[[ "hotel", "is_canceled"]]
average_cancels_by_hotel_df = cancelation_df.groupby("hotel").mean()
#average_cancels_by_hotel_df

# Graph City Hotel vs. Resort Hotel Average Cancellation Rates

In [53]:
#Bar graph of Cancelations of city hotel versus resort hotel
average_cancels_by_hotel_df.hvplot.bar( 
    xlabel="Name of Hotel",
    ylabel="Number of Cancelations",
    rot=0,
    title="City Hotel vs. Resort Hotel Cancellation Rate"
).opts(
    #yformatter='%.0f',
    color="blue",
    hover_color="yellow",
    ylim=(0, 0.5),
    xlim=(0,2)
)

c:\Users\jason\mambaforge\envs\devenv\Lib\site-packages\holoviews\core\util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
c:\Users\jason\mambaforge\envs\devenv\Lib\site-packages\holoviews\core\util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
c:\Users\jason\mambaforge\envs\devenv\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Bars   [hotel]   (is_canceled)

# What have we learned?  
The City Hotel experiences more cancelations, lets see if we can figure out what leads to these cancelations.

In [54]:
#creating dataframe for variables to pull from 
variables_of_cancels = hotel_booking_df[[ "hotel", "is_canceled", "INFLATION", "INFLATION_CHG", "FUEL_PRCS", "previous_cancellations", "previous_bookings_not_canceled",
                                         "booking_changes", "deposit_type", "reservation_status_date"]]
#variables_of_cancels.()

In [55]:
money_cancels = hotel_booking_df[["hotel", "INFLATION", "INFLATION_CHG", "FUEL_PRCS", ]]
behavior_cancels = hotel_booking_df[["hotel", "is_canceled", "previous_cancellations", "previous_bookings_not_canceled"]]

# Money Cancels are all variables that have to do with money.
-Inflation, Inflation Change, and Fuel Prices.
# Behavior Cancels are all variables that have to do with customer behavior. 
-Have they Cancelled(is_cancelled), Previous Cancels, Previous bookings not canceled.

In [56]:
money_cancels_df = money_cancels.groupby("hotel").mean()
behavior_cancels_df = behavior_cancels.groupby("hotel").mean()

In [57]:
#money_cancels_df

# Graph Money Cancel Variables

In [58]:
#Graph Money Cancels and Behavior Cancels
money_cancels_df.hvplot.bar( 
    xlabel="Name of Hotel",
    ylabel="Price",
    rot=30,
    title="City Hotel vs. Resort Hotel Money Cancels"
).opts(
    yformatter='%.0f',
    color="blue",
    hover_color="yellow",
    ylim=(0, 200),
    xlim=(0,2)
)

c:\Users\jason\mambaforge\envs\devenv\Lib\site-packages\holoviews\core\util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
c:\Users\jason\mambaforge\envs\devenv\Lib\site-packages\holoviews\core\util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
c:\Users\jason\mambaforge\envs\devenv\Lib\site-packages\holoviews\core\util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
c:\Users\jason\mambaforge\envs\devenv\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be tr

:Bars   [hotel,Variable]   (value)

# Results for Money Cancels
 The City Hotel leads in all categories including Inflation, Inflation Change, and Fuel Prices. We know that the City Hotel has the larger amount of cancellations, these monetary variables typically effect sales. With higher fuel prices comes a raise in the price of transportation to get to the hotel, this could be anything from plane ticket price, gas price, rental car price, and sometimes all of the above. The Resort Hotel is not too far behind on any of thes monetary variables, lets see how the behavioral cancels look before we make a decision.

# Graph Behavioral Cancels

In [59]:
#behavior cancels bar graph
behavior_cancels_df.hvplot.bar( 
    xlabel="Name of Hotel",
    ylabel="Amount",
    rot=30,
    title="City Hotel vs. Resort Hotel Behavioral Cancels"
).opts(
    yformatter='%.0f',
    color="blue",
    hover_color="yellow",
    ylim=(0, 1),
    xlim=(0,2)
)

c:\Users\jason\mambaforge\envs\devenv\Lib\site-packages\holoviews\core\util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
c:\Users\jason\mambaforge\envs\devenv\Lib\site-packages\holoviews\core\util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
c:\Users\jason\mambaforge\envs\devenv\Lib\site-packages\holoviews\core\util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
c:\Users\jason\mambaforge\envs\devenv\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be tr

:Bars   [hotel,Variable]   (value)

# Results for Behavioral Cancels
 - The Resort Hotel has a higher average in previous cancellations and previous bookings not canceled but has the lower average cancellation rate ot the two.
 - The City Hotel has a lower average in previous cancellations and previous bookings not canceled but has the higher average cancellation rate of the two.
-The City Hotel experiences larger numbers of cancels when looking at behavioral cancels compared to the Resort Hotel in which both numbers are all simular except for the average amount of cancellations. This shows us that we can not look at the amount the previous behavior of the customers to give us an idea on wether they will cancel again in the future or not.